In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [3]:
sys.path.append('/Vol0/user/f.konokhov/odometry/')

In [4]:
import sys
sys.path.append('/Vol0/user/f.konokhov/FlavorNet/utilities/slam_metrics')
sys.path.append('/Vol0/user/f.konokhov/FlavorNet/')

In [7]:
%%javascript
var command = "NOTEBOOK_PATH = '" + IPython.notebook.notebook_path + "'";
IPython.notebook.kernel.execute(command);

<IPython.core.display.Javascript object>

In [8]:
NOTEBOOK_PATH

'odometry/notebooks/retail_bot-train.ipynb'

In [9]:
from FlavorNet.models.train import set_notebook_id, set_experiment_id
notebook_id = set_notebook_id(NOTEBOOK_PATH)
experiment_id = set_experiment_id(notebook_id)

Using TensorFlow backend.


notebook_id = "notebooks-retail_bot-train"
experiment_id = "notebooks-retail_bot-train_2019-06-11-12:02:52"


In [10]:
!nvidia-smi

Tue Jun 11 15:03:04 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:04:00.0 Off |                  Off |
| N/A   37C    P0    50W / 250W |   1125MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:0D:00.0 Off |                  Off |
| N/A   32C    P0    49W / 250W |   7489MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [11]:
from FlavorNet.models.computation_utils import set_computation, fix_random_seed
set_computation(cuda_visible_devices=0)

['/job:localhost/replica:0/task:0/device:GPU:0']


In [162]:
target_size = (120,160)

x_col = ['path_to_optical_flow']
y_col = ['euler_x', 'euler_y', 'euler_z', 'x', 'y', 'z']
image_columns = ['path_to_optical_flow']
load_modes = ['flow_xy']
preprocess_modes = ['flow_xy']

In [163]:
from FlavorNet.models.config_utils import get_config
config = {
    "dataset_root": "/Vol0/user/f.konokhov/odometry/notebooks/",
    "csv_name": "df_new.csv",
    "train_sequences": [
        "meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned_tum/"
    ],
    "val_sequences": [
        "meetingroom_05_rgbd_ir_imu_pose_KITTI_finetuned_tum/"
        
        
    ],
    "depth_multiplicator": 0.001,
    "val_sampling_step": 1,
    "batch_size": 128
}


In [1]:
from FlavorNet.models.datasets import TUMDataset
dataset = TUMDataset(
    target_size=target_size,
    x_col=x_col,
    y_col=y_col,
    image_columns=image_columns,
    load_modes=load_modes,
    preprocess_modes=preprocess_modes,
    cached_imgs={},
    **config
)

NameError: name 'target_size' is not defined

In [ ]:
traj1_big = dataset.df_train[:int(len( dataset.df_train)/3)]
traj2_big = dataset.df_val[:int(len( dataset.df_val)/3)]
traj2_small = dataset.df_val[int(len( dataset.df_val)/3)::2]
traj1_small =dataset.df_train[int(len( dataset.df_train)/3)::2]
import pandas as pd
dataset.df_train = pd.concat([traj1_big ,traj2_big]).reset_index(drop=True)
dataset.df_val = pd.concat([traj1_small ,traj2_small]).reset_index(drop=True)

In [ ]:
traj1_big = dataset.df_train[:int(len( dataset.df_train)/3):2]
traj2_big = dataset.df_val[:int(len( dataset.df_val)/3):2]
traj2_small = dataset.df_val[int(len( dataset.df_val)/3):]
traj1_small =dataset.df_train[int(len( dataset.df_train)/3):]
import pandas as pd
dataset.df_train = pd.concat([traj1_small ,traj2_small]).reset_index(drop=True)
dataset.df_val = pd.concat([traj1_big ,traj2_big]).reset_index(drop=True)

In [139]:
from linalg.trajectory import RelativeTrajectory

In [140]:
gt_relative_trajectory = RelativeTrajectory()

In [141]:
keys = ['euler_x', 'euler_y', 'euler_z', 'x', 'y', 'z']


In [142]:
gt = traj1_big[keys].to_dict(orient='records')
gt_relative_trajectory.from_euler_angles(gt)
gt_global_trajectory = gt_relative_trajectory.relative_to_global()
from evaluation.visualization_utils import visualize_trajectory
visualize_trajectory(gt_global_trajectory, title='traj1_big', is_3d=True, file_name='gt_3d_traj1_big.html')

In [143]:
gt = traj1_small[keys].to_dict(orient='records')
gt_relative_trajectory.from_euler_angles(gt)
gt_global_trajectory = gt_relative_trajectory.relative_to_global()
from evaluation.visualization_utils import visualize_trajectory
visualize_trajectory(gt_global_trajectory, title='traj1_small', is_3d=True, file_name='gt_3d_traj1_small.html')

In [144]:
gt = traj2_big[keys].to_dict(orient='records')
gt_relative_trajectory.from_euler_angles(gt)
gt_global_trajectory = gt_relative_trajectory.relative_to_global()
from evaluation.visualization_utils import visualize_trajectory
visualize_trajectory(gt_global_trajectory, title='traj2_big', is_3d=True, file_name='gt_3d_traj2_big.html')

In [145]:
gt = traj2_small[keys].to_dict(orient='records')
gt_relative_trajectory.from_euler_angles(gt)
gt_global_trajectory = gt_relative_trajectory.relative_to_global()
from evaluation.visualization_utils import visualize_trajectory
visualize_trajectory(gt_global_trajectory, title='traj2_small', is_3d=True, file_name='gt_3d_traj2_small.html')

In [166]:
from FlavorNet.models.computation_utils import fix_random_seed
from FlavorNet.models.model_fabric import ModelFabric
from FlavorNet.models.train import train
seed=1
fix_random_seed(seed)

# model_fabric = ModelFabric(
#     target_size,
#     channels_counts=dataset.channels_counts,
#     lr=0.001,
#     loss='huber',
#     scale_rotation=50
# )
# model = model_fabric.construct_ls_vo_no_decoder_flexible_encoder_model(use_gated_convolutions=True,               
#                                  strides=[2, 1, 4, 1 ,2, 1],
#                                  dilation_rates=[1,1,1,1,1,1],
#                                  kernel_sizes = [7, 5, 3, 3, 3, 3])
from models.model_factory import ModelFactory

model_factory = ModelFactory(
    target_size,
    channels_counts=dataset.channels_counts,
    lr=0.001,
    loss='mae',
    scale_rotation=50
)
model = model_factory.construct_pretrained_model(
    '/Vol0/user/i.slynko/projects/odometry/notebooks/discoman_odometry.h5')
print(model.summary())

print('\n' + '='*98 + '\n\n')
print('SEED = {}'.format(seed))
print('\n' + '-'*98 + '\n')
#train(experiment_id, notebook_id, NOTEBOOK_PATH, model, dataset, period=10, num_epochs = 60)
#del model

Load weights from /Vol0/user/i.slynko/projects/odometry/notebooks/discoman_odometry.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 160, 2)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 80, 64)   6336        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 60, 80, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 60, 80, 64)   102464      activation_1[0][0]               
______________________

In [167]:
for layer in model.layers[:-6]:
    print(layer)

In [168]:
for layer in model.layers[:-6]:
    layer.trainable = False

In [169]:
train(experiment_id, notebook_id, NOTEBOOK_PATH, model, dataset, period=12, num_epochs = 100)

Set empty cache
Set empty cache
Set empty cache
Set empty cache
Epoch 1/100
48/48 [==============================] - 133s 3s/step - loss: 1.0559 - r_x_loss: 1.2730e-04 - r_y_loss: 0.0044 - r_z_loss: 0.0162 - t_x_loss: 0.0132 - t_y_loss: 7.7492e-04 - t_z_loss: 0.0058 - r_x_rmse: 3.1623e-04 - r_y_rmse: 0.0044 - r_z_rmse: 0.0162 - t_x_rmse: 0.0132 - t_y_rmse: 8.7747e-04 - t_z_rmse: 0.0058 - val_loss: 0.6912 - val_r_x_loss: 8.1398e-05 - val_r_y_loss: 0.0019 - val_r_z_loss: 0.0117 - val_t_x_loss: 0.0066 - val_t_y_loss: 3.8999e-04 - val_t_z_loss: 0.0042 - val_r_x_rmse: 3.1623e-04 - val_r_y_rmse: 0.0019 - val_r_z_rmse: 0.0117 - val_t_x_rmse: 0.0066 - val_t_y_rmse: 5.3999e-04 - val_t_z_rmse: 0.0042
Epoch 2/100
48/48 [==============================] - 4s 80ms/step - loss: 0.8726 - r_x_loss: 1.2053e-04 - r_y_loss: 9.0557e-04 - r_z_loss: 0.0162 - t_x_loss: 0.0101 - t_y_loss: 6.8078e-04 - t_z_loss: 0.0013 - r_x_rmse: 3.1637e-04 - r_y_rmse: 9.3865e-04 - r_z_rmse: 0.0162 - t_x_rmse: 0.0101 - t_y_rms

Epoch 13/100
48/48 [==============================] - 4s 82ms/step - loss: 0.8351 - r_x_loss: 1.0871e-04 - r_y_loss: 1.7828e-04 - r_z_loss: 0.0162 - t_x_loss: 0.0100 - t_y_loss: 6.7465e-04 - t_z_loss: 3.9450e-04 - r_x_rmse: 3.1623e-04 - r_y_rmse: 3.1790e-04 - r_z_rmse: 0.0162 - t_x_rmse: 0.0100 - t_y_rmse: 8.1469e-04 - t_z_rmse: 4.6141e-04 - val_loss: 0.6010 - val_r_x_loss: 6.8744e-05 - val_r_y_loss: 1.5509e-04 - val_r_z_loss: 0.0117 - val_t_x_loss: 0.0064 - val_t_y_loss: 3.8830e-04 - val_t_z_loss: 3.2460e-04 - val_r_x_rmse: 3.1623e-04 - val_r_y_rmse: 3.2541e-04 - val_r_z_rmse: 0.0117 - val_t_x_rmse: 0.0065 - val_t_y_rmse: 5.3897e-04 - val_t_z_rmse: 4.0435e-04
Epoch 14/100
48/48 [==============================] - 4s 80ms/step - loss: 0.8353 - r_x_loss: 1.0807e-04 - r_y_loss: 1.7168e-04 - r_z_loss: 0.0162 - t_x_loss: 0.0100 - t_y_loss: 6.8466e-04 - t_z_loss: 6.2397e-04 - r_x_rmse: 3.1623e-04 - r_y_rmse: 3.1735e-04 - r_z_rmse: 0.0162 - t_x_rmse: 0.0100 - t_y_rmse: 8.1808e-04 - t_z_rmse: 

Epoch 25/100
48/48 [==============================] - 4s 80ms/step - loss: 0.2164 - r_x_loss: 1.9236e-04 - r_y_loss: 2.0715e-04 - r_z_loss: 0.0038 - t_x_loss: 0.0075 - t_y_loss: 6.8668e-04 - t_z_loss: 4.3328e-04 - r_x_rmse: 3.4110e-04 - r_y_rmse: 3.6437e-04 - r_z_rmse: 0.0038 - t_x_rmse: 0.0075 - t_y_rmse: 8.1929e-04 - t_z_rmse: 5.1982e-04 - val_loss: 0.1669 - val_r_x_loss: 2.0924e-04 - val_r_y_loss: 1.4985e-04 - val_r_z_loss: 0.0028 - val_t_x_loss: 0.0069 - val_t_y_loss: 4.2214e-04 - val_t_z_loss: 4.4968e-04 - val_r_x_rmse: 3.1835e-04 - val_r_y_rmse: 3.4448e-04 - val_r_z_rmse: 0.0028 - val_t_x_rmse: 0.0069 - val_t_y_rmse: 5.4984e-04 - val_t_z_rmse: 5.4029e-04
Epoch 26/100
48/48 [==============================] - 4s 80ms/step - loss: 0.2008 - r_x_loss: 1.2433e-04 - r_y_loss: 1.4310e-04 - r_z_loss: 0.0036 - t_x_loss: 0.0064 - t_y_loss: 6.7615e-04 - t_z_loss: 4.3317e-04 - r_x_rmse: 3.2582e-04 - r_y_rmse: 3.4038e-04 - r_z_rmse: 0.0036 - t_x_rmse: 0.0064 - t_y_rmse: 8.1380e-04 - t_z_rmse: 

Epoch 37/100
48/48 [==============================] - 4s 80ms/step - loss: 0.1779 - r_x_loss: 9.0300e-05 - r_y_loss: 1.0570e-04 - r_z_loss: 0.0033 - t_x_loss: 0.0029 - t_y_loss: 6.7160e-04 - t_z_loss: 4.6962e-04 - r_x_rmse: 3.2121e-04 - r_y_rmse: 3.2599e-04 - r_z_rmse: 0.0033 - t_x_rmse: 0.0029 - t_y_rmse: 8.1396e-04 - t_z_rmse: 5.2740e-04 - val_loss: 0.1598 - val_r_x_loss: 3.7273e-05 - val_r_y_loss: 1.1440e-04 - val_r_z_loss: 0.0030 - val_t_x_loss: 0.0039 - val_t_y_loss: 3.8414e-04 - val_t_z_loss: 3.5102e-04 - val_r_x_rmse: 3.1623e-04 - val_r_y_rmse: 3.4276e-04 - val_r_z_rmse: 0.0030 - val_t_x_rmse: 0.0039 - val_t_y_rmse: 5.3737e-04 - val_t_z_rmse: 3.9882e-04
Epoch 38/100
48/48 [==============================] - 4s 80ms/step - loss: 0.1770 - r_x_loss: 8.9540e-05 - r_y_loss: 1.0535e-04 - r_z_loss: 0.0033 - t_x_loss: 0.0029 - t_y_loss: 6.6285e-04 - t_z_loss: 5.3938e-04 - r_x_rmse: 3.1917e-04 - r_y_rmse: 3.2272e-04 - r_z_rmse: 0.0033 - t_x_rmse: 0.0029 - t_y_rmse: 8.0432e-04 - t_z_rmse: 

Epoch 49/100
48/48 [==============================] - 4s 82ms/step - loss: 0.1686 - r_x_loss: 1.2417e-04 - r_y_loss: 1.0251e-04 - r_z_loss: 0.0031 - t_x_loss: 0.0025 - t_y_loss: 6.5883e-04 - t_z_loss: 2.3118e-04 - r_x_rmse: 3.1776e-04 - r_y_rmse: 3.2030e-04 - r_z_rmse: 0.0031 - t_x_rmse: 0.0026 - t_y_rmse: 8.0134e-04 - t_z_rmse: 3.6480e-04 - val_loss: 0.1508 - val_r_x_loss: 1.2546e-04 - val_r_y_loss: 7.6276e-05 - val_r_z_loss: 0.0027 - val_t_x_loss: 0.0039 - val_t_y_loss: 3.8545e-04 - val_t_z_loss: 4.0968e-04 - val_r_x_rmse: 3.1623e-04 - val_r_y_rmse: 3.3081e-04 - val_r_z_rmse: 0.0028 - val_t_x_rmse: 0.0039 - val_t_y_rmse: 5.3913e-04 - val_t_z_rmse: 4.2319e-04
Epoch 50/100
48/48 [==============================] - 4s 80ms/step - loss: 0.1704 - r_x_loss: 1.3107e-04 - r_y_loss: 9.9959e-05 - r_z_loss: 0.0031 - t_x_loss: 0.0025 - t_y_loss: 6.5525e-04 - t_z_loss: 4.1165e-04 - r_x_rmse: 3.2122e-04 - r_y_rmse: 3.2294e-04 - r_z_rmse: 0.0031 - t_x_rmse: 0.0025 - t_y_rmse: 8.0040e-04 - t_z_rmse: 

Epoch 61/100
48/48 [==============================] - 4s 81ms/step - loss: 0.1478 - r_x_loss: 4.7359e-05 - r_y_loss: 3.6509e-05 - r_z_loss: 0.0028 - t_x_loss: 0.0021 - t_y_loss: 6.4740e-04 - t_z_loss: 1.4187e-04 - r_x_rmse: 3.1682e-04 - r_y_rmse: 3.1722e-04 - r_z_rmse: 0.0028 - t_x_rmse: 0.0022 - t_y_rmse: 7.9530e-04 - t_z_rmse: 3.2031e-04 - val_loss: 0.1499 - val_r_x_loss: 2.3955e-05 - val_r_y_loss: 8.9590e-05 - val_r_z_loss: 0.0028 - val_t_x_loss: 0.0040 - val_t_y_loss: 3.8662e-04 - val_t_z_loss: 1.6844e-04 - val_r_x_rmse: 3.1623e-04 - val_r_y_rmse: 3.3637e-04 - val_r_z_rmse: 0.0028 - val_t_x_rmse: 0.0040 - val_t_y_rmse: 5.4019e-04 - val_t_z_rmse: 3.3817e-04
Epoch 62/100
48/48 [==============================] - 4s 81ms/step - loss: 0.1461 - r_x_loss: 3.4640e-05 - r_y_loss: 3.8989e-05 - r_z_loss: 0.0028 - t_x_loss: 0.0021 - t_y_loss: 6.4578e-04 - t_z_loss: 1.4186e-04 - r_x_rmse: 3.1704e-04 - r_y_rmse: 3.1838e-04 - r_z_rmse: 0.0028 - t_x_rmse: 0.0021 - t_y_rmse: 7.9267e-04 - t_z_rmse: 

Epoch 73/100
48/48 [==============================] - 4s 82ms/step - loss: 0.1374 - r_x_loss: 9.4416e-06 - r_y_loss: 2.6611e-05 - r_z_loss: 0.0027 - t_x_loss: 0.0019 - t_y_loss: 6.4205e-04 - t_z_loss: 4.8284e-05 - r_x_rmse: 3.1624e-04 - r_y_rmse: 3.1698e-04 - r_z_rmse: 0.0027 - t_x_rmse: 0.0020 - t_y_rmse: 7.9073e-04 - t_z_rmse: 3.1727e-04 - val_loss: 0.1495 - val_r_x_loss: 6.2590e-06 - val_r_y_loss: 7.7302e-05 - val_r_z_loss: 0.0028 - val_t_x_loss: 0.0039 - val_t_y_loss: 3.8138e-04 - val_t_z_loss: 6.6906e-05 - val_r_x_rmse: 3.1624e-04 - val_r_y_rmse: 3.3421e-04 - val_r_z_rmse: 0.0029 - val_t_x_rmse: 0.0039 - val_t_y_rmse: 5.3844e-04 - val_t_z_rmse: 3.3446e-04
Epoch 74/100
48/48 [==============================] - 4s 79ms/step - loss: 0.1364 - r_x_loss: 8.0331e-06 - r_y_loss: 2.3761e-05 - r_z_loss: 0.0026 - t_x_loss: 0.0019 - t_y_loss: 6.3854e-04 - t_z_loss: 5.0612e-05 - r_x_rmse: 3.1647e-04 - r_y_rmse: 3.1652e-04 - r_z_rmse: 0.0027 - t_x_rmse: 0.0020 - t_y_rmse: 7.8882e-04 - t_z_rmse: 

Epoch 85/100
48/48 [==============================] - 4s 81ms/step - loss: 0.1322 - r_x_loss: 4.7969e-06 - r_y_loss: 1.6636e-05 - r_z_loss: 0.0026 - t_x_loss: 0.0018 - t_y_loss: 6.3653e-04 - t_z_loss: 3.0087e-05 - r_x_rmse: 3.1624e-04 - r_y_rmse: 3.1635e-04 - r_z_rmse: 0.0026 - t_x_rmse: 0.0019 - t_y_rmse: 7.8741e-04 - t_z_rmse: 3.1688e-04 - val_loss: 0.1484 - val_r_x_loss: 4.9213e-06 - val_r_y_loss: 6.3224e-05 - val_r_z_loss: 0.0028 - val_t_x_loss: 0.0038 - val_t_y_loss: 3.8295e-04 - val_t_z_loss: 5.7440e-05 - val_r_x_rmse: 3.1625e-04 - val_r_y_rmse: 3.3247e-04 - val_r_z_rmse: 0.0028 - val_t_x_rmse: 0.0038 - val_t_y_rmse: 5.3944e-04 - val_t_z_rmse: 3.3478e-04
Epoch 86/100
48/48 [==============================] - 4s 80ms/step - loss: 0.1318 - r_x_loss: 4.0852e-06 - r_y_loss: 1.6591e-05 - r_z_loss: 0.0026 - t_x_loss: 0.0018 - t_y_loss: 6.4035e-04 - t_z_loss: 2.9498e-05 - r_x_rmse: 3.1623e-04 - r_y_rmse: 3.1638e-04 - r_z_rmse: 0.0026 - t_x_rmse: 0.0019 - t_y_rmse: 7.9130e-04 - t_z_rmse: 

Epoch 97/100
48/48 [==============================] - 4s 80ms/step - loss: 0.1307 - r_x_loss: 3.6887e-06 - r_y_loss: 1.5260e-05 - r_z_loss: 0.0025 - t_x_loss: 0.0018 - t_y_loss: 6.6036e-04 - t_z_loss: 2.6635e-05 - r_x_rmse: 3.1623e-04 - r_y_rmse: 3.1629e-04 - r_z_rmse: 0.0026 - t_x_rmse: 0.0019 - t_y_rmse: 8.1170e-04 - t_z_rmse: 3.1665e-04 - val_loss: 0.1487 - val_r_x_loss: 5.1812e-06 - val_r_y_loss: 6.3634e-05 - val_r_z_loss: 0.0028 - val_t_x_loss: 0.0038 - val_t_y_loss: 3.8370e-04 - val_t_z_loss: 5.2675e-05 - val_r_x_rmse: 3.1625e-04 - val_r_y_rmse: 3.3283e-04 - val_r_z_rmse: 0.0029 - val_t_x_rmse: 0.0039 - val_t_y_rmse: 5.4003e-04 - val_t_z_rmse: 3.3607e-04
Epoch 98/100
48/48 [==============================] - 4s 80ms/step - loss: 0.1302 - r_x_loss: 3.6782e-06 - r_y_loss: 1.5173e-05 - r_z_loss: 0.0025 - t_x_loss: 0.0018 - t_y_loss: 6.3580e-04 - t_z_loss: 2.6557e-05 - r_x_rmse: 3.1623e-04 - r_y_rmse: 3.1623e-04 - r_z_rmse: 0.0026 - t_x_rmse: 0.0019 - t_y_rmse: 7.8715e-04 - t_z_rmse: 